In [1]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/aishwaryamate/Datasets/main/Churn_Modelling.csv')

In [6]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [8]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [9]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [10]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [11]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [14]:
ann = Sequential()

ann.add(Dense(units = 10, activation = 'relu'))
ann.add(Dense(units = 1, activation = 'sigmoid'))

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6781 - accuracy: 0.5925
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6773 - accuracy: 0.5939
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6764 - accuracy: 0.5950
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6756 - accuracy: 0.5960
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6747 - accuracy: 0.5975
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.6738 - accuracy: 0.5993
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6730 - accuracy: 0.6005
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.6721 - accuracy: 0.6019
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6712 - accuracy: 0.6036
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.6703 - accura

In [15]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 [==============================] - 0s 1ms/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [16]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.78      0.88      0.83      1585
           1       0.14      0.07      0.09       415

    accuracy                           0.71      2000
   macro avg       0.46      0.48      0.46      2000
weighted avg       0.65      0.71      0.68      2000



In [17]:
ann.evaluate(xtrain,ytrain)

250/250 [==============================] - 1s 4ms/step - loss: 0.6049 - accuracy: 0.7206


[0.6048598289489746, 0.7206249833106995]

# Hyperparameter tuning


In [23]:
counter = 0

for i in range(0,3):
  if counter == 0:
    print('Python')
else:
    print('Machine learning')
counter = counter+1

Python
Python
Python
Machine learning


In [26]:
for i in range(0,3):
  print('Python')

print('Machine learning')

Python
Python
Python
Machine learning


In [21]:
counter

1

In [32]:
a = 10
type(a)

b = str(a)
type(b)
a = 'Python'
b = 'Batch'
a,b

a+' '+b

'Python Batch'

In [33]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [34]:
def hyper(hp): #This defines a function named hyper that takes a single argument hp.
    model = Sequential()
    counter = 0 #Initializes a counter variable to keep track of the current layer being added to the model.
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):  #Loops through a range of layers, where the number of layers is a hyperparameter named 'num_layers' with a minimum value of 1 and a maximum value of 10.
        if counter ==0:
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                            input_dim = 13)) #Adds the first layer with input dimensions specified by input_dim=13. The number of units ('units') and activation function ('activation') are hyperparameters, and their values are determined by the hp object.
        else:
          model.add(Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))  #Adds subsequent layers (after the first layer) with hyperparameter-specified units and activation functions.
    counter+=1   #Increments the counter to keep track of the layers added.
    model.add(Dense(units = 1,activation='sigmoid'))   #Adds the output layer with a single unit and sigmoid activation function, typical for binary classification problems.
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy',metrics = 'accuracy')
    return model  #Returns the compiled model.

In [35]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [36]:
tuner.search(xtrain,ytrain,epochs = 5, validation_data = (xtest,ytest))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.7925000190734863

Best val_accuracy So Far: 0.859000027179718
Total elapsed time: 00h 00m 19s


In [37]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units0': 64,
 'activation0': 'relu',
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'units2': 8,
 'activation2': 'relu',
 'units3': 8,
 'activation3': 'relu',
 'units4': 8,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'relu',
 'units6': 8,
 'activation6': 'relu',
 'units7': 8,
 'activation7': 'relu'}

In [38]:
tuned_model = tuner.get_best_models(num_models=1)[0]
# tuned_model.build(x.shape)

In [39]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 [==============================] - 3s 4ms/step - loss: 0.3445 - accuracy: 0.8570 - val_loss: 0.3465 - val_accuracy: 0.8600
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3374 - accuracy: 0.8608 - val_loss: 0.3376 - val_accuracy: 0.8630
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3366 - accuracy: 0.8631 - val_loss: 0.3477 - val_accuracy: 0.8550
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3318 - accuracy: 0.8620 - val_loss: 0.3365 - val_accuracy: 0.8605
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3300 - accuracy: 0.8633 - val_loss: 0.3437 - val_accuracy: 0.8590
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3274 - accuracy: 0.8639 - val_loss: 0.3451 - val_accuracy: 0.8630
Epoch 7/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3257 - accuracy: 0.8633 - val_loss: 0.3397 - val_accuracy: 0.8635
Epoch 

In [40]:
tuned_model.evaluate(xtrain,ytrain)

250/250 [==============================] - 1s 3ms/step - loss: 0.2778 - accuracy: 0.8844


[0.27776584029197693, 0.8843749761581421]

In [41]:
tuned_model.evaluate(xtest,ytest)

63/63 [==============================] - 0s 2ms/step - loss: 0.3647 - accuracy: 0.8570


[0.36473140120506287, 0.8569999933242798]